# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [70]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Check MPS availability using torch.backends.mps.is_available()
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Load dataset
dataset = load_dataset("climatebert", "climate_sentiment")

# Define labels
labels = list(dataset["train"].features["label"].names)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model_ft = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=len(labels))
model_lora = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=len(labels))

# Define LoRA parameters
lora_module = torch.nn.Linear(model_lora.base_model.wte.weight.shape[1], model_lora.base_model.wte.weight.shape[0])


def lora_parameters(module):
  if isinstance(module, torch.nn.Linear):
    yield module.weight


model_lora._modules["lora"] = lora_module
model_lora.lora.weight = torch.nn.Parameter(torch.randn(model_lora.base_model.wte.weight.shape[1], model_lora.base_model.wte.weight.shape[0]))


def get_lora_optimizer(model):
  return torch.optim.Adam([*lora_parameters(model), {"params": model.base_model.parameters(), "lr": 1e-5}])


def get_full_ft_optimizer(model):
  return torch.optim.Adam(model.parameters(), lr=2e-5)


def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = (labels == preds).sum().float() / len(labels)
  return {"accuracy": acc}


def train(model_name, model, optimizer, train_dataset, eval_dataset, epochs):
  training_args = TrainingArguments(
      output_dir=f"./results/{model_name}",
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=epochs,
      load_best_model_at_end=True,
      metric_for_best_model="accuracy",
      logging_steps=50,
      evaluation_strategy="epoch",
      fp16=True if device == "mps" else False,
      eval_dataset=eval_dataset,
      training_dataset=train_dataset,
      # Leverage MPS for training and evaluation
      gradient_accumulation_steps=2 if device == "mps" else 1,
      place="mps" if device == "mps" else device,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      optimizers=(optimizer, None),
  )

  trainer.train()
  return trainer.evaluate("eval")


# Define training functions
train_ft = lambda: train("full_ft", model_ft.to(device), get_full_ft_optimizer(model_ft), dataset["train"], dataset["test"], 3)
train_lora = lambda: train("lora", model_lora.to(device), get_lora_optimizer(model_lora), dataset["train"], dataset["test"], 3)


# Evaluate base model (untrained)
def evaluate_base():
  with torch.no_grad():
    inputs = tokenizer(dataset["test"], return_tensors=device)
    outputs = model_ft.to(device)(**inputs)
    preds = outputs.logits.argmax(-1)
    acc = (preds == dataset["test"]["label"]).sum().float() / len(preds)
    print(f"Base Model Accuracy: {acc}")


# Run Evaluations
evaluate_base()
print(train_ft())
print(train_lora())

# Inference Example
text = "Climate change presents a major risk to our planet."
inputs = tokenizer(text, return_tensors=device)
with torch.no_grad():
  outputs = model_ft


FileNotFoundError: Couldn't find a dataset script at /Users/andrewwrist/Documents/Projects/AI_ML/LoRA_climate_sentiment/climatebert/climatebert.py or any data file in the same directory. Couldn't find 'climatebert' on the Hugging Face Hub either: FileNotFoundError: Dataset 'climatebert' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.

In [71]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Check MPS availability using torch.backends.mps.is_available()
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [40]:
device

'mps'

In [72]:
# Load dataset
dataset = load_dataset("climatebert/climate_sentiment")

# Define labels
labels = list(dataset["train"].features["label"].names)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model_ft = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=len(labels))
model_lora = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=len(labels))

# Define LoRA parameters
lora_module = torch.nn.Linear(model_lora.base_model.wte.weight.shape[1], model_lora.base_model.wte.weight.shape[0])


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
def lora_parameters(module):
  if isinstance(module, torch.nn.Linear):
    yield module.weight


model_lora._modules["lora"] = lora_module
model_lora.lora.weight = torch.nn.Parameter(torch.randn(model_lora.base_model.wte.weight.shape[1], model_lora.base_model.wte.weight.shape[0]))



In [74]:
def get_lora_optimizer(model):
  return torch.optim.Adam([*lora_parameters(model), {"params": model.base_model.parameters(), "lr": 1e-5}])


def get_full_ft_optimizer(model):
  return torch.optim.Adam(model.parameters(), lr=2e-5)

In [75]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = (labels == preds).sum().float() / len(labels)
  return {"accuracy": acc}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [76]:
def train(model_name, model, optimizer, train_dataset, eval_dataset, epochs):
  training_args = TrainingArguments(
      output_dir=f"./results/{model_name}",
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=epochs,
      load_best_model_at_end=True,
      metric_for_best_model="accuracy",
      logging_steps=50,
      evaluation_strategy="epoch",
      fp16=True if device == "mps" else False,
      eval_dataset=eval_dataset,
      training_dataset=train_dataset,
      # Leverage MPS for training and evaluation
      gradient_accumulation_steps=2 if device == "mps" else 1,
      place="mps" if device == "mps" else device,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      optimizers=(optimizer, None),
  )

  trainer.train()
  return trainer.evaluate("eval")

In [77]:
# Define training functions
train_ft = lambda: train("full_ft", model_ft.to(device), get_full_ft_optimizer(model_ft), dataset["train"], dataset["test"], 1)
train_lora = lambda: train("lora", model_lora.to(device), get_lora_optimizer(model_lora), dataset["train"], dataset["test"], 1)


In [78]:
def evaluate_base():
  with torch.no_grad():
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(dataset["test"]["text"], return_tensors="pt", padding=True)
    outputs = model_ft(**inputs)  # Remove device specification for automatic MPS handling
    preds = outputs.logits.argmax(-1)
    acc = (preds == dataset["test"]["label"]).sum().float() / len(preds)
    print(f"Base Model Accuracy: {acc}")


In [79]:
evaluate_base()

: 

: 

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [37]:
import timeit
import torch
import random
x = torch.ones(50000000, device = 'mps')
timeit.timeit(lambda: x * random.randint(0,100), number=1)

0.06786775001091883

In [38]:
x = torch.ones(50000000, device = 'cpu')
timeit.timeit(lambda: x * random.randint(0,100), number=1)

0.04576800001086667

In [39]:
x = torch.ones(50000000, device = 'mps')
timeit.timeit(lambda: x * random.randint(0,100), number=1)

0.00019916699966415763